In [1]:
import numpy as np
import torch
import argparse
from pathlib import Path
import pickle

In [2]:
par_dir = Path('/Users/fammasmaz/Downloads/sncf_random_test_201/')

In [3]:
# import the pickle file
with open(par_dir/'postpro.pkl', 'rb') as f:
    postpro = pickle.load(f)

In [4]:
stacked = postpro[3]

In [5]:
stacked.shape

(66106, 26)

In [6]:
# information on what is what
# 0: contact id
# 1: contact candidate
# 2: contact antagoniste
# 3-5: rl
# 6-8: vl
# 9: gapTT
# 10-12: coordinates
# 13-15: local frame n
# 16-18: local frame t
# 19-21: local frame s
# 22: contact type
# 23-25: intercenter vectors


In [7]:
stacked[1]


array([ 2.00000000e+00,  3.24000000e+02,  5.83000000e+02,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -2.49967720e-03,  1.20314920e-02,
       -1.43559947e-02,  6.55085339e-05, -5.91363856e-01, -9.76515191e-01,
        5.63191351e-02,  1.00000000e+00, -0.00000000e+00,  1.77351632e-01,
       -9.84147549e-01,  4.67504311e-01,  8.69977371e-01,  1.56777209e-01,
        8.83990792e-01, -4.60093222e-01, -8.29126523e-02, -5.91363856e-01,
       -9.76515191e-01,  5.63191351e-02])

In [1]:
from utilities.postpro_utilities import mass_writer, coordinates_writer, dof_writer, arvd_writer, read_pickled_file


In [2]:
out_dir = '/Users/fammasmaz/Downloads/sncf_random_test_201/'

In [3]:
bodies = coordinates_writer(out_dir)
dof = dof_writer(out_dir)
arvd = arvd_writer(out_dir)
mass = mass_writer(out_dir)

ValueError: cannot reshape array of size 164064 into shape (10)

In [ ]:
dof = read_pickled_file(out_dir, 'dof.pkl')
coord = read_pickled_file(out_dir, 'coordinates.pkl')
arvd = read_pickled_file(out_dir, 'arvd.pkl')
mass = read_pickled_file(out_dir, 'mass.pkl')


In [ ]:
dof.shape, coord.shape, arvd.shape, mass.shape

((10254, 10), (10254, 3), (10254, 2), (10253, 1))

In [ ]:
dof

array([['1.000000e+00', '0.000000e+00', '0.000000e+00', ...,
        '0.000000e+00', '0.000000e+00', '0.000000e+00'],
       ['2.000000e+00', '2.719124e-02', '4.875036e-02', ...,
        '1.482859e-04', '-3.512882e-05', '3.019887e-03'],
       ['3.000000e+00', '2.665231e-02', '2.120281e-01', ...,
        '8.138976e-03', '2.238133e-03', '-4.967885e-03'],
       ...,
       ['1.025200e+04', '-3.503255e-03', '4.907533e-03', ...,
        '1.039922e-05', '-2.830651e-05', '3.088422e-05'],
       ['1.025300e+04', '-6.467959e-03', '-4.327955e-02', ...,
        '-1.106675e-05', '-3.381411e-05', '-7.738222e-06'],
       ['1.025400e+04', '-2.135540e-02', '7.760525e-03', ...,
        '3.629142e-05', '-7.868708e-05', '-3.986986e-06']], dtype='<U32')

In [ ]:
dof[1]

array(['2.000000e+00', '2.719124e-02', '4.875036e-02', '-4.244340e+00',
       '5.012035e-05', '-1.497262e-05', '-4.645571e-05', '1.482859e-04',
       '-3.512882e-05', '3.019887e-03'], dtype='<U32')

In [29]:
# writing the postpro script
import numpy as np
from utilities.postpro_utilities import mass_writer, coordinates_writer, dof_writer, arvd_writer, read_pickled_file, plate_connection

def node_creator(out_dir):
    # dof x(3), v(3), a (6)
    # coord x, y, z
    # arvd 1
    # mass 1

    coordinates_writer(out_dir)
    dof_writer(out_dir)
    arvd_writer(out_dir)
    mass_writer(out_dir)
    dof = read_pickled_file(out_dir, 'dof.pkl')
    coord = read_pickled_file(out_dir, 'coordinates.pkl')
    arvd = read_pickled_file(out_dir, 'arvd.pkl')
    mass = read_pickled_file(out_dir, 'mass.pkl')
    x = np.hstack((dof, coord, arvd, mass))
    x = x.astype(np.float32)
    return x

def edge_index_creator(out_dir):
  # information on what is what
  # 0: contact id
  # 1: contact candidate
  # 2: contact antagoniste
  # 3-5: rl
  # 6-8: vl
  # 9: gapTT
  # 10-12: coordinates
  # 13-15: local frame n
  # 16-18: local frame t
  # 19-21: local frame s
  # 22: contact type
  # 23-25: intercenter vectors
  postpro = read_pickled_file(out_dir, 'postpro.pkl', False)
  edge_features = postpro[3]
  edge_index = np.array(edge_features[:,1:3]).T
  edge_features = edge_features[:,3:]

  return edge_index.astype(np.float32), edge_features.astype(np.float32)

def hot_vector_plate_conection(out_dir, n_nodes):
   plated = plate_connection(out_dir)
   x = np.zeros((n_nodes, 1))
   x[plated-1] = 1
   x[0] = 1
   return x.astype(np.float32)


def gravitational_force_creator(m, g=9.8):
    f = np.zeros((m.shape[0], 3))
    # first one is rigid plate
    f[1:, 2] = -m[1:, 0]*g
    return f

x = node_creator(out_dir)
n_nodes = x.shape[0]
edge_index, edge_features = edge_index_creator(out_dir)
n = hot_vector_plate_conection(out_dir, n_nodes)
f = gravitational_force_creator(x[:, -1].reshape(-1, 1))
y = x.copy()


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [4]:
x.shape, edge_index.shape, edge_features.shape, n.shape

((10254, 22), (2, 66106), (66106, 23), (6327, 1))

In [39]:
# find the names of the files that start with 'sncf'
from pathlib import Path

par_dir = Path('/Users/fammasmaz/Downloads/')
stem = 'sncf_random_test_2'
files = [f for f in par_dir.iterdir() if f.stem.startswith(stem)]

In [40]:
files

[PosixPath('/Users/fammasmaz/Downloads/sncf_random_test_201'),
 PosixPath('/Users/fammasmaz/Downloads/sncf_random_test_2')]